# How to Render Design into Ansys

## 1. Perform the necessary imports and create a design in Metal

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [4]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

QAnsysRenderer.default_options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'ansys_file_extension': '.aedt',
 'bounding_box_scale_x': 1.2,
 'bounding_box_scale_y': 1.2}

In [5]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

In [6]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs.

In [7]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '5.7 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2a. Render into Ansys HFSS
Ansys COM registration occurs automatically.

#### Before running the following cells:
Manually open a new Ansys project with a HFSS eigenmode design. Here is how:

1. Launch `ANSYS Electronics Desktop yyyy Rx`.
2. Create a new Ansys project by clicking on the `New` icon at the top left.
3. Create a new HFSS design inside that project by navigating to the menu `Project` and selecting `Insert HFSS Design`.
4. Change the HFSS design to eigenmode by right-clicking on the HFSSdesign1 that just got created inside your project (left panel) and then selecting: `Solution Type...`. Select Eigenmode.

In [8]:
fourq_hfss = design.renderers.hfss # Access Ansys HFSS renderer

In [9]:
fourq_hfss.open_ansys_design() # Open either a new or existing design based on default options

ModuleNotFoundError: No module named 'pythoncom'

NOTE: If you need to change the active Ansys HFSS design linked to Metal, first select the correct project in the Ansys GUI by double clicking on it in the leftmost panel. Then re-run the two cells above.

If the Ansys HFSS design is not empty, run the following cell to clear it. This is equivalent to doing the following from the Ansys GUI: hold down Ctrl + A (to select everything in the design) -> right clicking -> Edit -> Delete.

In [ ]:
obj_names = fourq_hfss.pinfo.get_all_object_names()
if obj_names:
    fourq_hfss.clean_active_design()

#### Render some component

In [ ]:
fourq_hfss.render_design([], []) # Render the whole Metal design into Ansys HFSS.
# fourq_hfss.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')]) # Render a single qubit with 2 endcaps into Ansys HFSS.
# fourq_hfss.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')]) # Render 2 qubits and 2 endcaps, one per qubit, into Ansys HFSS.

Before trying rendering a different combination of components, clear the design by executing the following cell

In [ ]:
obj_names = fourq_hfss.pinfo.get_all_object_names()
if obj_names:
    fourq_hfss.clean_active_design()

## 2b. Render into Ansys Q3D
Ansys COM registration occurs automatically.

#### Before running the following cells, setup Ansys Q3D manually:
Manually open a new Ansys project with a Q3D design. Here is how:

1. Launch `ANSYS Electronics Desktop yyyy Rx`.
2. Crate a new Ansys project by clicking on the `New` icon at the top left.
3. Create a new Q3D design inside that project by navigating to the menu `Project` and selecting `Insert Q3D Extractor Design`.

In [ ]:
fourq_q3d = design.renderers.q3d # Access Ansys Q3D renderer

In [ ]:
fourq_q3d.open_ansys_design() # Open either a new or existing design based on default options

NOTE: If you need to change the active Ansys Q3D design linked to Metal, first select the correct project in the Ansys GUI by double clicking on it in the leftmost panel. Then re-run the two cells above.

If the Ansys Q3D design is not empty, run the following cell to clear it. This is equivalent to doing the following from the Ansys GUI: hold down Ctrl + A (to select everything in the design) -> right clicking -> Edit -> Delete.

In [ ]:
obj_names = fourq_q3d.pinfo.get_all_object_names()
if obj_names:
    fourq_q3d.clean_active_design()

#### Render some component

In [ ]:
fourq_q3d.render_design([], []) # Render the whole Metal design into Ansys Q3D.
# fourq_q3d.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')]) # Render a single qubit with 2 endcaps into Ansys Q3D.
# fourq_q3d.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')]) # Render 2 qubits and 2 endcaps, one per qubit, into Ansys Q3D.

## 3. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix
This follows from step 2b above.

In [ ]:
fourq_q3d.add_q3d_setup() # Add a solution setup.

In [ ]:
fourq_q3d.analyze_setup("Setup") # Analyze said solution setup.

In [ ]:
fourq_q3d.get_capacitance_matrix() # Get its capacitance matrix.

If objects are in the project, you can delete them.

In [ ]:
obj_names = fourq_q3d.pinfo.get_all_object_names()
if obj_names:
    fourq_q3d.clean_active_design()

  self.draw()



## References - Miscellaneous pyEPR/Ansys commands
The following commands are for reference only to better understand how the backend code works. They're not meant to be run directly in this notebook as part of the guide.

import pyEPR as epr

Connect to Ansys directly from notebook:

pinfo = epr.ProjectInfo(project_path = None, 
                        project_name = None,
                        design_name  = None)
modeler = pinfo.design.modeler

Access methods within HfssDesign class in pyEPR:

epr.ansys.HfssDesign.create_dm_setup
epr.ansys.HfssDesign.create_q3d_setup

Get project and design names:

pinfo.project_name
design._design.GetName()

Filter qgeometry table:

full_table = design.qgeometry.tables['poly']
mask = full_table['subtract'] == False
table = full_table[mask]

Draw centered rectangles:

bigsquare = modeler.draw_rect_center([0, 0, 0], x_size=8, y_size=8, name='bigsquare')
topright = modeler.draw_rect_center([2, 2, 0], x_size=2, y_size=2, name='topright')

Subtracting shapes:

modeler.subtract('bigsquare', ['topright'])

Draw centered box:

modeler.draw_box_center([0, 0, 0], [1, 2, 3])

Draw closed polygon:

trianglepts = [[-1, 5, 0], [1, 5, 0], [0, 7, 0]]
modeler.draw_polyline(trianglepts, closed=True)

Draw polyline:

smallpts = [[2.85, 0, 0], [3.15, 0, 0]]
modeler.draw_polyline(smallpts, closed=False)

Sweep one polyline with another:

modeler._sweep_along_path('Polyline8', 'Polyline7')